In [1]:
# pip install seaborn

In [1]:
%matplotlib qt5
import pyaccel
import pymodels
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as scyint
import pandas as pd
from mathphys import constants
from mathphys.functions import save_pickle, load_pickle
from mathphys.beam_optics import beam_rigidity

import touschek_pack.functions as tousfunc
import touschek_pack.Classes as tousclass


In [2]:
acc = pymodels.si.create_accelerator()
acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(acc)

tousan = tousclass.Tous_analysis(acc)
spos = pyaccel.lattice.find_spos(acc, indices='closed')


In [3]:
# vchamber = [-0.008,0.008, -0.003, 0.003]
# # dic = tousan.get_reordered_dict(spos[10:30], 'lost_positions', scrap=True, vchamber=vchamber)

In [4]:
# change_scrap = np.linspace(0.004, 0.01, 6)
# dec = np.sort(change_scrap)[::-1]

# for idx, iten in enumerate(dec):
#     vchamber = [-iten, iten, -(iten*0.4), iten*0.4]
#     dic = tousan.get_reordered_dict(spos, 'lost_positions', scrap=True, vchamber=vchamber)
#     var = save_pickle(dic, fname='pic_{}'.format(idx))

In [5]:
# dic = load_pickle('/home/thaleseqb/repos/pic_0.pickle')

In [6]:
# tousan.plot_scat_table(spos, dic,1,1)


In [7]:
# fam = pymodels.si.get_family_data(acc)
# great = []
# for lista in fam['ID']['index']:
#     for iten in lista:
#         great.append(iten)


In [8]:
# plt.figure()
# plt.scatter(spos[great], np.zeros(len(great)))
# pyaccel.graphics.draw_lattice(acc, offset=0, height=0.01, gca=True)
# plt.show()

In [9]:
# tousan.get_lost_profile(dic)

# twi,*_ = pyaccel.optics.calc_twiss(acc, indices='closed')
# beta = twi.betax
# plt.figure()
# plt.plot(spos, beta)
# pyaccel.graphics.draw_lattice(acc, offset=0, height=1, gca=True)

In [10]:
# # indices = pyaccel.lattice.find_indices(acc, 'fam_name', 'B2')
# fam_data = pymodels.si.get_family_data(acc)


In [11]:
# s = spos
# tousan.plot_scat_table(s, n_c=1, n_r=1)


In [12]:
# indices = fam_data['BC']['index']

# for index_list in indices:
#     init, final = index_list[0], index_list[-1]
#     s = spos[init:final]
#     tousan.plot_scat_table(s, n_c=1, n_r=1)


In [13]:
tousan.get_amps_idxs()

(True,
 (array([-0.04372908, -0.04372908, -0.04372908, ..., -0.04372908,
         -0.04372908, -0.04372908]),
  array([0.04372908, 0.04372908, 0.04372908, ..., 0.04372908, 0.04372908,
         0.04372908])),
 (array([0.00000000e+00, 1.00036646e-01, 2.00073292e-01, ...,
         5.18189827e+02, 5.18289863e+02, 5.18389900e+02]),
  array([0.04372908, 0.04372908, 0.04372908, ..., 0.04372908, 0.04372908,
         0.04372908]),
  array([-0.04372908, -0.04372908, -0.04372908, ..., -0.04372908,
         -0.04372908, -0.04372908])))

In [14]:
accep = tousan.accep
accep

(array([-0.04372908, -0.04372908, -0.04372908, ..., -0.04372908,
        -0.04372908, -0.04372908]),
 array([0.04372908, 0.04372908, 0.04372908, ..., 0.04372908, 0.04372908,
        0.04372908]))

In [15]:
tousan.plot_analysis_at_position(180,'pos',accep)

In [7]:
ltime = pyaccel.lifetime.Lifetime(acc)

rate = ltime.touschek_data['rate']


(array([0.00000000e+00, 1.00036646e-01, 2.00073292e-01, ...,
        5.18189827e+02, 5.18289863e+02, 5.18389900e+02]),
 array([0.04372908, 0.04372908, 0.04372908, ..., 0.04372908, 0.04372908,
        0.04372908]),
 array([-0.04372908, -0.04372908, -0.04372908, ..., -0.04372908,
        -0.04372908, -0.04372908]))

In [18]:
scalc, acpep,acepn = tousan.s_calc

In [20]:
ltime = pyaccel.lifetime.Lifetime(acc)
rate = ltime.touschek_data['rate']

fig, ax = plt.subplots(figsize=(10,5))

ax.plot(scalc, rate)

pyaccel.graphics.draw_lattice(acc, offset=- 6 *1e-7, height=1e-6, gca=True)
ax.set_xlabel('s [m]', fontsize=16)
ax.set_ylabel('Touschek scattering rate [1/s]', fontsize=16)
ax.tick_params(axis='both', labelsize=14)
plt.tight_layout()
ax.set_xlim(0,50)
plt.show()

In [14]:


pyaccel.lattice.find_indices(acc, 'pass_method','drift_pass')
tousan.get_amps_idxs()
tousan.plot_normtousd(np.array([26, 52, 78]))

[2,
 6,
 7,
 9,
 12,
 15,
 17,
 19,
 22,
 25,
 27,
 66,
 69,
 71,
 73,
 75,
 77,
 80,
 83,
 129,
 132,
 134,
 136,
 138,
 141,
 144,
 146,
 190,
 192,
 194,
 196,
 198,
 200,
 202,
 205,
 208,
 254,
 257,
 259,
 261,
 263,
 265,
 268,
 307,
 309,
 312,
 315,
 317,
 319,
 321,
 324,
 325,
 326,
 328,
 329,
 331,
 332,
 334,
 336,
 337,
 340,
 342,
 344,
 346,
 349,
 352,
 354,
 393,
 396,
 398,
 400,
 402,
 404,
 407,
 410,
 456,
 459,
 461,
 463,
 465,
 468,
 471,
 473,
 517,
 519,
 521,
 523,
 525,
 527,
 529,
 532,
 535,
 581,
 584,
 586,
 588,
 590,
 592,
 595,
 634,
 636,
 639,
 642,
 644,
 646,
 648,
 651,
 652,
 653,
 654,
 655,
 658,
 659,
 660,
 661,
 662,
 665,
 667,
 669,
 671,
 674,
 677,
 679,
 718,
 721,
 723,
 725,
 727,
 729,
 732,
 735,
 781,
 784,
 786,
 788,
 790,
 793,
 796,
 798,
 842,
 844,
 846,
 848,
 850,
 852,
 854,
 857,
 860,
 906,
 909,
 911,
 913,
 915,
 917,
 920,
 959,
 961,
 964,
 967,
 969,
 971,
 973,
 976,
 977,
 978,
 980,
 981,
 983,
 984,
 986,
 98

In [7]:
model=pymodels.si.create_accelerator()
model.cavity_on=False
model.radiation_on=False
model.vchamber_on=False
off = 0
off1 = 0.02
off2 = -0.02
twi,*_ = pyaccel.optics.calc_twiss(model, energy_offset=off, indices='closed')

twi1,*_ = pyaccel.optics.calc_twiss(model, energy_offset=off1, indices='closed')
twi2,*_ = pyaccel.optics.calc_twiss(model, energy_offset=off2, indices='closed')

betax,betax1,betax2 = twi.betax, twi1.betax,twi2.betax
etax, rx = twi.etax, twi.rx
rx1, rx2 = twi1.rx, twi2.rx


In [8]:
# fig, (ax,ay)=plt.subplots(1,2, sharey=True, sharex=True)


# ax.plot(spos, betay, label= r'$\beta z (\delta$ = {}%)'.format(off*1e2))


# ay.plot(spos, betay1, label= r'$\beta z (\delta$ = {}%)'.format(off1*1e2))

# ax.legend(fontsize=18, loc='best')


# ax.set_ylabel(r'$\beta$ [m]', fontsize=25)
# ax.set_xlabel('s [m]', fontsize=25)
# pyaccel.graphics.draw_lattice(model, offset=0, height=1, gca=ax)

# ay.legend(fontsize=18, loc='best')
# ay.set_xlabel('s [m]', fontsize=25)
# pyaccel.graphics.draw_lattice(model, offset=0, height=1, gca=ay)
# ax.tick_params(axis='both', labelsize=18)
# ay.tick_params(axis='both', labelsize=18)



In [76]:
fig = plt.figure(figsize=(13,7))
gs = plt.GridSpec(3, 1, hspace=0.02, top=0.85, bottom=0.11, left=0.07, right=0.98, height_ratios=[3,1,3])
ax = fig.add_subplot(gs[0, 0])
az = fig.add_subplot(gs[1, 0], sharex=ax)
ay = fig.add_subplot(gs[2, 0], sharex=ax)

ax.plot(spos, betax2, label= r'{}%'.format(off2*1e2))
ax.plot(spos, betax, label= r'{}%'.format(off*1e2))
ax.plot(spos, betax1, label=r'{}%'.format(off1*1e2))

ay.plot(spos, (rx2-rx)*1e3/off2, label= r'$\delta$ = {}%'.format(off2*1e2))
ay.plot(spos, etax*1e3, label= r'$\delta$ = {}%'.format(off*1e2))
ay.plot(spos, (rx1-rx)*1e3/off1, label= r'$\delta$ = {}%'.format(off1*1e2))

ax.legend(
    fontsize=18, title=r'$\delta$', title_fontsize=24,
    loc='lower center', bbox_to_anchor=[0.5,1.0], ncol=3)
az.set_axis_off()
az.set_ylim(-1.1, 1.1)

ax.set_ylabel(r'$\beta_x$ [m]', fontsize=25)
pyaccel.graphics.draw_lattice(model, offset=0, height=1, gca=az)

plt.setp(ax.get_xticklabels(), visible=False)

ay.set_xlabel('s [m]', fontsize=25)
ay.set_ylabel(r'$\eta_x$ [mm]', fontsize=25)
ax.set_xlim(0,50)
ax.tick_params(axis='both', labelsize=18)
ay.tick_params(axis='both', labelsize=18)
ax.grid(True, ls='--', lw=1, alpha=0.5)
ay.grid(True, ls='--', lw=1, alpha=0.5)
fig.show()

In [25]:
fig, ax = plt.subplots(1,1)

ax.plot(spos, accep[0]*1e2, label= 'neg. accep.')
ax.plot(spos, accep[1]*1e2, label= 'pos. accep.')

ax.legend(loc='best', fontsize=14)
pyaccel.graphics.draw_lattice(model, offset=0, height=1, gca=True)

ax.set_ylabel('energy acceptance [%]', fontsize=25)
ax.set_xlabel('s [m]', fontsize=25)
ax.tick_params(axis='both', labelsize=18)

plt.show()

In [3]:
s = [12.44, 6.86, 5.51, 7.32]

In [4]:
tousan.plot_histograms(s) #dasa

0.9998999557336539
0.999662824401865
0.9999543733046768
0.9996146431041102
